In [140]:
import numpy as np
import GeometricTools
import torch
from glob import glob

In [141]:
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull

In [142]:
def downsample_array(original_array, desired_shape):
    # Get the number of rows in the original array
    num_rows_original = original_array.shape[0]
    
    # Generate random indices to select 'b' rows
    random_indices = np.random.choice(num_rows_original, desired_shape[0], replace=False)
    
    # Select rows based on the random indices
    downsampled_array = original_array[random_indices]
    
    return downsampled_array

In [143]:
# from sklearn.metrics.pairwise import euclidean_distances
# from sklearn.neighbors import NearestNeighbors

# def fps_downsample(original_array, desired_shape):
#     # Calculate pairwise Euclidean distances
#     dist_matrix = euclidean_distances(original_array, original_array)
    
#     # Initialize an array to store selected indices
#     selected_indices = [0]
    
#     # Calculate remaining points using FPS
#     while len(selected_indices) < desired_shape[0]:
#         distances_to_selected = dist_matrix[:, selected_indices]
#         min_distances = np.min(distances_to_selected, axis=1)
#         max_min_distance_index = np.argmax(min_distances)
#         selected_indices.append(max_min_distance_index)
    
#     # Select points based on the selected indices
#     downsampled_array = original_array[selected_indices]
    
#     return downsampled_array

In [144]:
# def ccw(A, B, C):
#     return (B[0] - A[0]) * (C[1] - A[1]) - (C[0] - A[0]) * (B[1] - A[1]) > 0

# # Trả về bao lồi với thứ tự các điểm được liệt kê theo chiều kim đồng hồ
# def convexHull(p):
#     # Sắp xếp các điểm theo tọa độ x, nếu bằng nhau sắp xếp theo y
#     p = p[np.lexsort((p[:, 1], p[:, 0]))]

#     # Tập bao lồi
#     hull = [p[0]]

#     # Dựng bao trên
#     for i in range(1, len(p)):
#         while len(hull) >= 2 and ccw(hull[-2], hull[-1], p[i]):
#             hull.pop()
#         hull.append(p[i])

#     # Tiếp tục dựng bao dưới
#     for i in range(len(p) - 2, -1, -1):
#         while len(hull) >= 2 and ccw(hull[-2], hull[-1], p[i]):
#             hull.pop()
#         hull.append(p[i])

#     # Xoá đỉểm đầu được lặp lại ở cuối
#     if len(p) > 1:
#         hull.pop()
        
    

#     return hull

In [145]:
def interpolate_points(p1, p2, num_points=100):
    # Ensure p1 and p2 are NumPy arrays
    p1 = np.array(p1)
    p2 = np.array(p2)

    # Interpolate points between p1 and p2 using np.linspace
    interpolated_points = np.linspace(p1, p2, num_points + 2)[1:-1]


    return interpolated_points

In [146]:
paths = glob('np_points/*')
for path in paths:
    points = np.load(path)
    GeometricTools.drawPointCloudsColorsClasses( torch.tensor(points),  origins=[[0,0,0]]) 
    max_z = np.max(points[:,2])
    
    projection = points[:,:2] # take only x, y ( n, 2)
    desired_shape =  (2000,2)    #(int(projection.shape[0]/2000), 2)
    projection = downsample_array(projection, desired_shape)
    
    # ConvexHux for Detect smallest boundary
    hull = ConvexHull(projection)
    sorted_order = hull.vertices

    projection1 = projection[sorted_order]    
    x_coords = projection1[:, 0]
    y_coords = projection1[:, 1]

    boundary = projection1[0].reshape(1,2)
    
    projection1 = np.concatenate((projection1, boundary))
    
    for i in range(0,len(projection1)-1):  # create points for the boundary
        first = projection1[i]
        second = projection1[i+1]
        result = interpolate_points(first, second)
        boundary = np.concatenate((boundary, result))
    
    # create z for wall
    new_z = np.linspace(0,max_z, 100)
    repeated_array = np.repeat(boundary, len(new_z), axis=0)
    repeated_z = np.tile(new_z, len(boundary))
    
    new_array = np.column_stack((repeated_array, repeated_z))
    #GeometricTools.drawPointCloudsColorsClasses( torch.tensor(new_array),  origins=[[0,0,0]]) 
    
    # create floor and ceiling
    wall = np.copy(new_array)
    
    desired_shape =  (4000,2)   
    new_array = downsample_array(new_array, desired_shape)
    
    new_array = new_array[np.lexsort((new_array[:, 1], new_array[:, 0]))][:,:2]
    
    boundary_2 = new_array[0].reshape(1,2)
    
    for i in range(0,len(new_array)-1):
        first = new_array[i]
        second = new_array[i+1]
        result = interpolate_points(first, second)
        boundary_2 = np.concatenate((boundary_2, result))
        
    z_floor = np.zeros((boundary_2.shape[0],1))
    print(z_floor.shape)
    print(boundary_2.shape)
    floor = np.column_stack((boundary_2, z_floor))

    z_ceiling = np.ones((boundary_2.shape[0],1)) * max_z
    ceiling = np.column_stack((boundary_2, z_ceiling))
    room = np.concatenate((floor, ceiling, wall))
    
    GeometricTools.drawPointCloudsColorsClasses( torch.tensor(room),  origins=[[0,0,0]]) 
        

(399901, 1)
(399901, 2)
(399901, 1)
(399901, 2)
(399901, 1)
(399901, 2)


KeyboardInterrupt: 

In [20]:
import numpy as np

def interpolate_points(p1, p2, num_points=100):
    # Ensure p1 and p2 are NumPy arrays
    p1 = np.array(p1)
    p2 = np.array(p2)

    # Interpolate points between p1 and p2 using np.linspace
    interpolated_points = np.linspace(p1, p2, num_points + 2)[1:-1]

    return interpolated_points

# Example usage:
p1 = np.array([1, 1])  # First point
p2 = np.array([5, 5])  # Second point
grid_points = interpolate_points(p1, p2, num_points=100)
print(grid_points)

[[1.03960396 1.03960396]
 [1.07920792 1.07920792]
 [1.11881188 1.11881188]
 [1.15841584 1.15841584]
 [1.1980198  1.1980198 ]
 [1.23762376 1.23762376]
 [1.27722772 1.27722772]
 [1.31683168 1.31683168]
 [1.35643564 1.35643564]
 [1.3960396  1.3960396 ]
 [1.43564356 1.43564356]
 [1.47524752 1.47524752]
 [1.51485149 1.51485149]
 [1.55445545 1.55445545]
 [1.59405941 1.59405941]
 [1.63366337 1.63366337]
 [1.67326733 1.67326733]
 [1.71287129 1.71287129]
 [1.75247525 1.75247525]
 [1.79207921 1.79207921]
 [1.83168317 1.83168317]
 [1.87128713 1.87128713]
 [1.91089109 1.91089109]
 [1.95049505 1.95049505]
 [1.99009901 1.99009901]
 [2.02970297 2.02970297]
 [2.06930693 2.06930693]
 [2.10891089 2.10891089]
 [2.14851485 2.14851485]
 [2.18811881 2.18811881]
 [2.22772277 2.22772277]
 [2.26732673 2.26732673]
 [2.30693069 2.30693069]
 [2.34653465 2.34653465]
 [2.38613861 2.38613861]
 [2.42574257 2.42574257]
 [2.46534653 2.46534653]
 [2.5049505  2.5049505 ]
 [2.54455446 2.54455446]
 [2.58415842 2.58415842]


In [2]:
import numpy as np

# A -> B -> C đi theo thứ tự ngược chiều kim đồng hồ
def ccw(A, B, C):
    return (B[0] - A[0]) * (C[1] - A[1]) - (C[0] - A[0]) * (B[1] - A[1]) > 0

# Trả về bao lồi với thứ tự các điểm được liệt kê theo chiều kim đồng hồ
def convexHull(p):
    # Sắp xếp các điểm theo tọa độ x, nếu bằng nhau sắp xếp theo y
    p = p[np.lexsort((p[:, 1], p[:, 0]))]

    # Tập bao lồi
    hull = [p[0]]

    # Dựng bao trên
    for i in range(1, len(p)):
        while len(hull) >= 2 and ccw(hull[-2], hull[-1], p[i]):
            hull.pop()
        hull.append(p[i])

    # Tiếp tục dựng bao dưới
    for i in range(len(p) - 2, -1, -1):
        while len(hull) >= 2 and ccw(hull[-2], hull[-1], p[i]):
            hull.pop()
        hull.append(p[i])

    # Xoá đỉểm đầu được lặp lại ở cuối
    if len(p) > 1:
        hull.pop()

    return hull

if __name__ == "__main__":
    # n = int(input())
    # p = np.zeros((n, 2), dtype=int)
    # for i in range(n):
    #     p[i] = np.array(input().split(), dtype=int)
    p = np.array([[1,2], [2,6], [0,1], [6,6]])

    hull = convexHull(p)

    print(len(hull))
    for point in hull:
        print(point[0], point[1])


3
0 1
2 6
6 6
